<a href="https://colab.research.google.com/github/YS-JEOUNG/amazon-review-project/blob/main/code/artscrafts_code/artscrafts_bow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import pandas as pd
import numpy as np

import re, string, unicodedata
import nltk
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize, regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer, PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
!pip install contractions

     |████████████████████████████████| 266kB 4.3MB/s 
     |████████████████████████████████| 327kB 16.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85389 sha256=149b3f25db3f4da552f743414feb7bb833e75ecd07442df4e122a4f8c24a60e8
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [ ]:
from contractions import contractions_dict     # CONTRACTION_MAP 대신 사용

# Data

In [ ]:
total = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/arts_total.csv')

# Text Preprocessing

In [ ]:
# calculate raw tokens in order to measure of cleaned tokens
raw_tokens = len([w for t in (total['review_text'].apply(word_tokenize)) for w in t])
print(f'number of raw tokens: {raw_tokens}')

number of raw tokens: 11737017


In [ ]:
total['review_text']

0         Nice and perfect for an address book Thin, you...
1         Solid buy, few small gripes about page layout ...
2         Cool place to Catalog Your Recipes I purchased...
3         Its ok read for specifications I did not like ...
4         Great for Chefs. I'm a chef and I was looking ...
                                ...                        
130128     Five Stars Awesome set to have for any croceter.
130129    Variations in thickness with various colors on...
130130    super happy! Is really small but that's what I...
130131    Most excellent quality sock yarn on the market...
130132    a yarn that makes perfect socks. I finished 2 ...
Name: review_text, Length: 130133, dtype: object

In [ ]:
def strip_html(text):
  # html코드 제거 함수
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
  # 괄호로 묶여있는 단어 제거
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
  # 위에 함수 두개 과정 진행하는 함수
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text


# Define function to expand contractions
def expand_contractions(text):
  # IGNORECASE =  대소문자 관련 없이, DOTALL = 줄바꿈기호 상관없이 매칭해달라는 의미
  # re를 이용하여 정규표현식으로 매칭을 할 때 contractions_dict의 키를   
  # 찾고자 하는 형태를 만든뒤 매칭하여 축약문들을 확장 시키는 과정인듯
    contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_dict.get(match)\
                        if contractions_dict.get(match)\
                        else contractions_dict.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
    
    try:
      expanded_text = contractions_pattern.sub(expand_match, text)
      expanded_text = re.sub("'", "", expanded_text)
    except:
      return text
    return expanded_text


# special_characters removal
def remove_special_characters(text, remove_digits=True):
  # 문자, 숫자가 아닌것들을 제거
  # r은 정규 표현식을 쓸 때 가독성을 위한 장치 raw string이라는 의미
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words


def remove_punctuation_and_splchars(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_word = remove_special_characters(new_word, True)
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

stopword_list= stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
#stopword_list.remove('headphone')
#stopword_list.remove('headphones')
#stopword_list.remove('earbuds')
#stopword_list.remove('bud')
#stopword_list.remove('ear')
#stopword_list.remove('sony')
#stopword_list.remove('product')


def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopword_list:
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation_and_splchars(words)
    words = remove_stopwords(words)
    return words

def lemmatize(words):
    lemmas = lemmatize_verbs(words)
    return lemmas

In [ ]:
def normalize_and_lemmatize(input):
    sample = denoise_text(input)
    sample = expand_contractions(sample)
    sample = remove_special_characters(sample)
    words = nltk.word_tokenize(sample)
    words = normalize(words)
    lemmas = lemmatize(words)
    return ' '.join(lemmas)

In [ ]:
total['clean_text'] = total['review_text'].map(lambda text: normalize_and_lemmatize(text))

In [ ]:
total['clean_text']

0         nice perfect address book thin could slide dra...
1         solid buy small grip page layout honest alread...
2         cool place catalog recipes purchase recipe jou...
3         ok read specifications not like really like mo...
4         great chefs chef look book jot quick ideas ins...
                                ...                        
130128                       five star awesome set croceter
130129    variations thickness various color x little lo...
130130            super happy really small look super happy
130131    excellent quality sock yarn market use regia s...
130132    yarn make perfect sock finish pair sock years ...
Name: clean_text, Length: 130133, dtype: object

In [ ]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130133 entries, 0 to 130132
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   overall           130133 non-null  float64
 1   vote              130133 non-null  int64  
 2   verified          130133 non-null  bool   
 3   reviewer_id       130133 non-null  object 
 4   asin              130133 non-null  object 
 5   style             130133 non-null  object 
 6   reviewer_name     130128 non-null  object 
 7   unix_review_time  130133 non-null  int64  
 8   category          130133 non-null  object 
 9   description       130133 non-null  object 
 10  title             130133 non-null  object 
 11  brand             129770 non-null  object 
 12  rank              130133 non-null  object 
 13  main_cat          129791 non-null  object 
 14  date              20908 non-null   object 
 15  price             115105 non-null  object 
 16  review_text       13

In [ ]:
# cleaning the text
clean_tokens = len([w for t in (total['clean_text'].apply(word_tokenize)) for w in t])
print(f'number of clean tokens: {clean_tokens}')
print(f'percentage of removed tokens: {1-(clean_tokens/raw_tokens)}')

number of clean tokens: 5260371
percentage of removed tokens: 0.5518136337367493


In [ ]:
total.to_csv('/content/drive/MyDrive/Colab Notebooks/project/arts_total_cleaned.csv', sep=',', encoding='utf-8', index=False)